## VectorSearch with QdrantVectorStore – 검색 패턴

### 학습 목표

1. **기본 kNN 검색**: 가장 유사한 k개의 문서 검색
2. **Threshold 검색**: 임계값 이상의 유사도를 가진 문서만 검색
3. **Filter 검색**: 메타데이터 조건을 만족하는 문서 내에서 검색
4. **Hybrid 검색**: Dense + Sparse 벡터를 결합한 RRF 기반 검색

### 사전 준비

이 섹션을 실행하기 전에 앞선 섹션에서 다음이 완료되어야 합니다:
- Qdrant 컬렉션 생성 및 데이터 인덱싱
- Dense/Sparse Named Vectors 설정


In [2]:
import json
import os
import sys
import time
import warnings
from collections import Counter
from pathlib import Path
from pprint import pprint

import numpy as np
import pandas as pd
from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.models import (
    Distance,
    FieldCondition,
    Filter,
    MatchValue,
    PointStruct,
    QueryRequest,
    SearchRequest,
    SparseIndexParams,
    SparseVector,
    SparseVectorParams,
    VectorParams,
)

In [3]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    kwargs: dict[str, object] = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
    }
    if max_tokens is not None:
        kwargs["max_tokens"] = max_tokens
    if base_url:
        kwargs["base_url"] = base_url
    return ChatOpenAI(**kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs: object,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict[str, object] = {}
    embedding_kwargs: dict[str, object] = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: str = "float",
    input_text: str | list[str] = "",
    **kwargs: object,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [5]:
# Qdrant 클라이언트 생성 및 연결 확인
qdrant_url = "http://localhost:6333"
print(f"Qdrant URL: {qdrant_url}")

# Qdrant 클라이언트 생성
try:
    client = QdrantClient(url=qdrant_url)

    # 연결 확인
    collections = client.get_collections()
    if collections.collections:
        print("\n기존 Qdrant Collections 목록:")
        for col in collections.collections:
            info = client.get_collection(col.name)
            print(f"  - {col.name}: {info.points_count} points")

except Exception as e:
    print(f"Qdrant 연결 실패: {e}")
    client = None

Qdrant URL: http://localhost:6333

기존 Qdrant Collections 목록:
  - day2_kiwi_bm25_real_test: 8 points
  - day2_product: 0 points
  - day2_productionize_sample_collection: 0 points
  - day2_int8: 0 points
  - hybrid_search: 0 points
  - ranking_test: 5 points
  - multi_vector_demo: 3 points
  - day2_kiwi_bm25_hybrid: 0 points
  - chonkie_e2e_pipeline: 9 points
  - law_docs_v1: 3 points
  - mem0migrations: 1 points
  - docs_te3l: 0 points
  - chonkie_demo: 4 points
  - mem0_user: 3 points


In [9]:
# ============================================================================
# 1. 기본 kNN 검색 (k-Nearest Neighbors)
# ============================================================================


def qdrant_knn_search(
    query: str,
    collection_name: str = "demo_collection",
    k: int = 5,
    vector_name: str = "dense",
) -> list[tuple[Document, float]]:
    """
    기본 kNN 검색: 쿼리와 가장 유사한 상위 k개의 문서를 반환합니다.

    Args:
        query: 검색 쿼리
        collection_name: Qdrant 컬렉션 이름
        k: 반환할 문서 개수
        vector_name: 사용할 Named Vector 이름

    Returns:
        (Document, score) 튜플의 리스트
    """
    # Qdrant 클라이언트 연결
    client = QdrantClient(url=os.getenv("QDRANT_URL", "http://localhost:6333"))

    # Embeddings 초기화
    embeddings = OpenAIEmbeddings(
        model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
    )

    # VectorStore 생성
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
        vector_name=vector_name,
        retrieval_mode=RetrievalMode.DENSE,
    )

    # 유사도 검색 (기본 kNN)
    results = vectorstore.similarity_search_with_score(query, k=k)

    return results


# 테스트 예시
print("=" * 80)
print("1. 기본 kNN 검색")
print("=" * 80)

# 예제 쿼리
example_query = "연차휴가는 어떻게 계산되나요?"

try:
    results = qdrant_knn_search(
        query=example_query,
        k=3,
    )

    print(f"\n쿼리: {example_query}")
    print(f"검색 결과 (상위 {len(results)}개):\n")

    for idx, (doc, score) in enumerate(results, 1):
        print(f"[{idx}] 유사도 점수: {score:.4f}")
        print(f"    내용: {doc.page_content[:100]}...")
        print(f"    메타데이터: {doc.metadata}")
        print()

except Exception as e:
    print(f"검색 실패: {e}")
    print("이 셀을 실행하려면 먼저 Qdrant 컬렉션을 생성하고 데이터를 인덱싱해야 합니다.")

1. 기본 kNN 검색
검색 실패: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `demo_collection` doesn\'t exist!"},"time":0.000017125}'
이 셀을 실행하려면 먼저 Qdrant 컬렉션을 생성하고 데이터를 인덱싱해야 합니다.


In [10]:
# ============================================================================
# 2. Threshold 검색 (Score Threshold)
# ============================================================================


def qdrant_threshold_search(
    query: str,
    collection_name: str = "demo_collection",
    k: int = 10,
    score_threshold: float = 0.7,
    vector_name: str = "dense",
) -> list[tuple[Document, float]]:
    """
    Threshold 검색: 임계값 이상의 유사도를 가진 문서만 반환합니다.

    낮은 품질의 검색 결과를 필터링하여 정확도를 높입니다.

    Args:
        query: 검색 쿼리
        collection_name: Qdrant 컬렉션 이름
        k: 최대 반환 문서 개수
        score_threshold: 최소 유사도 점수 (0.0 ~ 1.0)
        vector_name: 사용할 Named Vector 이름

    Returns:
        (Document, score) 튜플의 리스트 (score >= threshold인 것만)
    """
    client = QdrantClient(url=os.getenv("QDRANT_URL", "http://localhost:6333"))
    embeddings = OpenAIEmbeddings(
        model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small"),
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
        vector_name=vector_name,
        retrieval_mode=RetrievalMode.DENSE,
    )

    # Retriever로 변환하여 threshold 설정
    retriever = vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": k,
            "score_threshold": score_threshold,
        },
    )

    # 검색 실행
    docs = retriever.invoke(query)

    # 점수와 함께 반환 (메타데이터에서 추출)
    results = []
    for doc in docs:
        score = doc.metadata.get("_score", 1.0)
        results.append((doc, score))

    return results


# 테스트 예시
print("=" * 80)
print("2. Threshold 검색")
print("=" * 80)

example_query = "퇴직금 계산 방법"

try:
    # 높은 임계값으로 검색
    high_threshold_results = qdrant_threshold_search(
        query=example_query,
        k=10,
        score_threshold=0.8,  # 높은 임계값
    )

    # 낮은 임계값으로 검색
    low_threshold_results = qdrant_threshold_search(
        query=example_query,
        k=10,
        score_threshold=0.5,  # 낮은 임계값
    )

    print(f"\n쿼리: {example_query}\n")

    print(f"높은 임계값 (≥0.8): {len(high_threshold_results)}개 문서")
    for idx, (doc, score) in enumerate(high_threshold_results[:3], 1):
        print(f"  [{idx}] 점수: {score:.4f} | {doc.page_content[:80]}...")

    print(f"\n낮은 임계값 (≥0.5): {len(low_threshold_results)}개 문서")
    for idx, (doc, score) in enumerate(low_threshold_results[:3], 1):
        print(f"  [{idx}] 점수: {score:.4f} | {doc.page_content[:80]}...")

    print("\n임계값이 높을수록 정확도↑, 재현율↓")
    print(" 임계값이 낮을수록 재현율↑, 정확도↓")

except Exception as e:
    print(f"검색 실패: {e}")

2. Threshold 검색
검색 실패: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `demo_collection` doesn\'t exist!"},"time":0.000018417}'


In [12]:
# ============================================================================
# 3. Filter 검색 (Metadata Filtering)
# ============================================================================


def qdrant_filter_search(
    query: str,
    filter_conditions: dict,
    collection_name: str = "demo_collection",
    k: int = 5,
    vector_name: str = "dense",
) -> list[tuple[Document, float]]:
    """
    Filter 검색: 메타데이터 조건을 만족하는 문서 내에서만 검색합니다.

    예: 특정 문서 타입, 날짜 범위, 태그 등으로 사전 필터링

    Args:
        query: 검색 쿼리
        filter_conditions: 필터 조건 딕셔너리
            예: {"source": "근로기준법", "pages_gte": 10}
        collection_name: Qdrant 컬렉션 이름
        k: 반환할 문서 개수
        vector_name: 사용할 Named Vector 이름

    Returns:
        (Document, score) 튜플의 리스트
    """
    from qdrant_client.models import FieldCondition, Filter, MatchValue, Range

    client = QdrantClient(url=os.getenv("QDRANT_URL", "http://localhost:6333"))
    embeddings = OpenAIEmbeddings(
        model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
    )

    # 쿼리 임베딩 생성
    query_vector = embeddings.embed_query(query)

    # 필터 조건 빌드
    must_conditions = []

    for key, value in filter_conditions.items():
        if key.endswith("_gte"):
            # Range: greater than or equal
            field = key[:-4]
            must_conditions.append(FieldCondition(key=field, range=Range(gte=value)))
        elif key.endswith("_lte"):
            # Range: less than or equal
            field = key[:-4]
            must_conditions.append(FieldCondition(key=field, range=Range(lte=value)))
        elif key.endswith("_gt"):
            # Range: greater than
            field = key[:-3]
            must_conditions.append(FieldCondition(key=field, range=Range(gt=value)))
        elif isinstance(value, (list, tuple)):
            # Multiple match values (OR)
            from qdrant_client.models import MatchAny

            must_conditions.append(FieldCondition(key=key, match=MatchAny(any=list(value))))
        else:
            # Exact match
            must_conditions.append(FieldCondition(key=key, match=MatchValue(value=value)))

    qdrant_filter = Filter(must=must_conditions) if must_conditions else None

    # 검색 실행
    search_result = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        using=vector_name,
        query_filter=qdrant_filter,
        limit=k,
        with_payload=True,
    )

    # Document 객체로 변환
    results = []
    for point in search_result.points:
        doc = Document(
            page_content=point.payload.get("page_content", ""),
            metadata=point.payload.get("metadata", {}),
        )
        results.append((doc, point.score))

    return results


# 테스트 예시
print("=" * 80)
print("3. Filter 검색")
print("=" * 80)

example_query = "근로시간 제한"

try:
    # 예시 1: 특정 소스 문서만 검색
    filter1 = {"source": "근로기준법"}
    results1 = qdrant_filter_search(
        query=example_query,
        filter_conditions=filter1,
        k=3,
    )

    print(f"\n쿼리: {example_query}")
    print(f"필터: {filter1}")
    print(f"결과: {len(results1)}개\n")

    for idx, (doc, score) in enumerate(results1, 1):
        print(f"[{idx}] 점수: {score:.4f}")
        print(f"    내용: {doc.page_content[:80]}...")
        print(f"    소스: {doc.metadata.get('source', 'N/A')}")
        print()

    # 예시 2: 페이지 범위 필터
    filter2 = {"pages_gte": 5}
    results2 = qdrant_filter_search(
        query=example_query,
        filter_conditions=filter2,
        k=3,
    )

    print("\n필터: 5페이지 이상 문서")
    print(f"결과: {len(results2)}개\n")

    # 예시 3: 복합 필터 (AND 조건)
    filter3 = {"source": "근로기준법", "ocr_used": True}
    results3 = qdrant_filter_search(
        query=example_query,
        filter_conditions=filter3,
        k=3,
    )

except Exception as e:
    print(f"⚠️  검색 실패: {e}")

3. Filter 검색
⚠️  검색 실패: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `demo_collection` doesn\'t exist!"},"time":0.000080083}'


In [4]:
# ============================================================================
# 4. Hybrid 검색 (Dense + Sparse RRF Fusion)
# ============================================================================


def reciprocal_rank_fusion(
    results_list: list[list[tuple[Document, float]]],
    k: int = 60,
) -> list[tuple[Document, float]]:
    """
    Reciprocal Rank Fusion (RRF): 여러 검색 결과를 `순위 기반으로 결합`합니다.

    RRF Score = Σ (1 / (k + rank_i))

    Args:
        results_list: 각 검색기의 결과 리스트
        k: RRF 상수 (기본값 60, 일반적으로 60이 최적)

    Returns:
        융합된 결과 (Document, fused_score) 리스트
    """
    # 문서별 점수 누적
    doc_scores = {}
    doc_objects = {}

    for results in results_list:
        for rank, (doc, _score) in enumerate(results, start=1):
            # 문서 식별자 생성 (content 기반 해시)
            doc_id = hash(doc.page_content)

            # RRF 점수 계산: 1 / (k + rank)
            rrf_score = 1.0 / (k + rank)

            if doc_id not in doc_scores:
                doc_scores[doc_id] = 0.0
                doc_objects[doc_id] = doc

            doc_scores[doc_id] += rrf_score

    # 점수 기준 정렬
    sorted_docs = sorted(
        doc_scores.items(),
        key=lambda x: x[1],
        reverse=True,
    )

    # 결과 리스트 생성
    fused_results = [(doc_objects[doc_id], score) for doc_id, score in sorted_docs]

    return fused_results


def qdrant_hybrid_search(
    query: str,
    collection_name: str = "demo_collection",
    k: int = 10,
    dense_vector_name: str = "dense",
    sparse_vector_name: str = "sparse",
    rrf_k: int = 60,
) -> list[tuple[Document, float]]:
    """
    Hybrid 검색: Dense와 Sparse 벡터 검색 결과를 RRF로 결합합니다.

    Args:
        query: 검색 쿼리
        collection_name: Qdrant 컬렉션 이름
        k: 각 검색에서 가져올 문서 개수
        dense_vector_name: Dense Named Vector 이름
        sparse_vector_name: Sparse Named Vector 이름
        rrf_k: RRF 상수

    Returns:
        RRF로 융합된 결과 리스트
    """
    client = QdrantClient(url=os.getenv("QDRANT_URL", "http://localhost:6333"))
    embeddings = OpenAIEmbeddings(
        model=os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
    )

    # 1. Dense 검색
    vectorstore_dense = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
        vector_name=dense_vector_name,
        retrieval_mode=RetrievalMode.DENSE,
    )
    dense_results = vectorstore_dense.similarity_search_with_score(query, k=k)

    # 2. Sparse 검색 (Kiwi 토크나이징)
    try:
        from kiwipiepy import Kiwi

        kiwi = Kiwi()

        # 쿼리 토크나이징
        tokens = [token.form for token in kiwi.tokenize(query)]

        # Sparse 벡터 생성 (간단한 TF 기반)
        from collections import Counter

        token_counts = Counter(tokens)

        # Sparse 검색 수행 (실제 구현 시 SPLADE 등 사용)
        # 여기서는 Dense만 사용 (Sparse는 Named Vector 설정 필요)
        sparse_results = []

        print(" Sparse 검색을 위해서는 Named Vector 'sparse' 설정이 필요합니다.")
        print(f"   토크나이징 결과: {tokens[:10]}")

    except ImportError:
        print("⚠️  kiwipiepy가 설치되지 않았습니다. Dense 검색만 수행합니다.")
        sparse_results = []

    # 3. RRF Fusion
    if sparse_results:
        fused_results = reciprocal_rank_fusion(
            [dense_results, sparse_results],
            k=rrf_k,
        )
    else:
        # Sparse 결과가 없으면 Dense만 반환
        fused_results = dense_results

    return fused_results


# 테스트 예시
print("=" * 80)
print("4. Hybrid 검색 (RRF Fusion)")
print("=" * 80)

example_query = "야간근로 수당 지급 기준"

try:
    results = qdrant_hybrid_search(
        query=example_query,
        k=5,
    )

    print(f"\n쿼리: {example_query}")
    print(f"결과 (RRF 융합): {len(results)}개\n")

    for idx, (doc, score) in enumerate(results[:5], 1):
        print(f"[{idx}] RRF 점수: {score:.6f}")
        print(f"    내용: {doc.page_content[:100]}...")
        print()

    print("\n" + "=" * 80)
    print("RRF (Reciprocal Rank Fusion) 개념")
    print("=" * 80)
    print("""
RRF는 여러 검색 결과를 순위 기반으로 결합하는 알고리즘입니다.

핵심 아이디어:
- 각 검색기의 순위(rank)를 역수로 변환하여 점수화
- 공식: score = 1 / (k + rank)
- k는 일반적으로 60 (경험적 최적값)

장점:
- 검색기별 점수 스케일이 달라도 정규화 불필요
- 상위 순위 문서에 더 높은 가중치
- 구현이 간단하고 효과적

예시:
Dense 검색: [Doc A(rank=1), Doc B(rank=2), Doc C(rank=3)]
Sparse 검색: [Doc B(rank=1), Doc C(rank=2), Doc D(rank=3)]

RRF 점수 (k=60):
- Doc A: 1/(60+1) = 0.0164
- Doc B: 1/(60+1) + 1/(60+1) = 0.0328  ← 두 검색 모두 상위
- Doc C: 1/(60+3) + 1/(60+2) = 0.0317
- Doc D: 1/(60+3) = 0.0159

최종 순위: B > C > A > D
""")

except Exception as e:
    print(f" 검색 실패: {e}")

4. Hybrid 검색 (RRF Fusion)
 검색 실패: name 'QdrantClient' is not defined


---

# Qdrant Named Vectors 실습: Dense + Sparse 하이브리드 인덱싱

## [Qdrant Named Vectors란](https://qdrant.tech/documentation/concepts/vectors/#named-vectors)

**단일 포인트에 여러 벡터를 저장**할 수 있는 기능입니다.
서로 다른 크기와 유형의 여러 벡터를 동일한 데이터 포인트에 저장할 수 있습니다.  
이는 서로 다른 특징이나 모달리티(예: 이미지, 텍스트 또는 비디오)를 표현하기 위해 여러 임베딩으로 데이터를 정의해야 할 때 유용할 수 있습니다.  
컬렉션 내에 별도의 명명된 벡터 공간을 생성해야 합니다.  
이러한 벡터 공간은 컬렉션 생성 시 정의할 수 있으며 독립적으로 관리할 수 있습니다.


### 사용 사례
- **Dense + Sparse**: 의미 검색 + 키워드 검색 동시 지원
- **Multi-Vector (ColBERT 등, Matryoseka 기법 응용)**: 여러 토큰 벡터 저장
- **Multi-Modal**: 텍스트 벡터 + 이미지 벡터 동시 저장

### 장점
1. 단일 컬렉션에서 여러 검색 전략 사용
2. Prefetch API로 결과 결합 (RRF Fusion)
3. 메타데이터 중복 없이 효율적 저장


In [5]:
# ============================================================================
# Named Vectors 컬렉션 생성 (Dense + Sparse)
# ============================================================================
from qdrant_client.models import (
    HnswConfigDiff,
    PayloadSchemaType,
    ScalarQuantization,
    ScalarQuantizationConfig,
    ScalarType,
)

# 컬렉션명
COLLECTION_NAME = "hybrid_search"

# 기존 컬렉션 삭제
if client:
    if client.collection_exists(COLLECTION_NAME):
        client.delete_collection(COLLECTION_NAME)
        print("  - 기존 컬렉션 삭제 완료")

    # Named Vectors 설정
    # 1. Dense vector (의미 검색)
    # 2. Sparse vector (키워드 검색)
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={
            "dense": VectorParams(
                size=1024,  # KURE-v1(BGE-M3 기반)
                distance=Distance.COSINE,
                on_disk=False,  # 프로덕션 최적화: 디스크 기반
            ),
            # NOTE: 여기에 이제 여러가지 벡터의 Naming 및 파라미터를 선언해줄 수 있습니다.
        },
        sparse_vectors_config={
            "sparse": SparseVectorParams(
                index=SparseIndexParams(
                    on_disk=False,
                )
            ),
        },
        # NOTE: [참고 문서](https://qdrant.tech/documentation/guides/optimize/)
        quantization_config=ScalarQuantization(
            scalar=ScalarQuantizationConfig(
                type=ScalarType.INT8,
                quantile=0.99,
                always_ram=True,
            )
        ),
        hnsw_config=HnswConfigDiff(
            m=16,  # 정확도 vs 속도 균형
            ef_construct=100,
            full_scan_threshold=10000,
        ),
    )

    print(f"컬렉션 생성 완료: {COLLECTION_NAME}")

    # Payload 인덱싱 **메타데이터 필터 성능 향상 - DB Indexing 과 동일하다고 생각하시면 됩니다.**
    client.create_payload_index(
        collection_name=COLLECTION_NAME,
        field_name="source",
        field_schema=PayloadSchemaType.KEYWORD,
    )

    client.create_payload_index(
        collection_name=COLLECTION_NAME,
        field_name="chunk_id",
        field_schema=PayloadSchemaType.INTEGER,
    )

## Notebook 01에서 저장한 청크 데이터 로드
# TODO: 환경에 맞춰서 저장했던 경로에 대한 수정이 필요할 수 있습니다.
DATA_DIR = DAY2_ROOT / "data"
chunks_file = DATA_DIR / "chunks.json"
chunks_data = {}

# 청크 데이터 로드
if chunks_file.exists():
    print(f"청크 파일 발견: {chunks_file}")

    with open(chunks_file, encoding="utf-8") as f:
        chunks_data = json.load(f)

    print(f"청크 로드 완료: {len(chunks_data)}개")

    # 샘플 출력
    if chunks_data:
        print("\n첫 번째 청크 샘플:")
        print(f"  - 텍스트: {chunks_data[0].get('text', '')}...")
        print(f"  - 메타데이터: {chunks_data[0].get('metadata', {})}")

print(f"\n현재 청크 수: {len(chunks_data)}개")

NameError: name 'client' is not defined